# Azure ML SDK for jobs
Sample to leverage Azure ML SDK to define job and execute.

In [ ]:
subscription_id = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
resource_group = "RESOURCE_GROUP_NAME"
workspace = "WORKSPACE_NAME"
region = "REGION"

In [ ]:
# Define the workspace client

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION=subscription_id
RESOURCE_GROUP=resource_group
WS_NAME=workspace

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

# Verify that the handle works correctly.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

## AML Compute Target
AML jobs execute at compute cluster level, which is scale accordingly, and the compute cluster is sharable with other users. There's a queue system at the backend to execute the jobs being submitted.

In [ ]:
# Try to retrieve AML compute, and create AML compute if the compute cluster is not available

from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "ws-aml-cpu-01"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=2
    )
    ml_client.compute.begin_create_or_update(compute).result()

## AML Environment
To execute the job, the code will be executed in AML Environment, which is a container image executed in.

More information here: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-environments-v2?view=azureml-api-2&tabs=cli

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/curated/sklearn-1.5:21",
    conda_file="./conda-yamls/pydata.yaml",
    name="workshop-conda-env",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

In [ ]:
# Define MLFlow tracking URI
mlflow_tracking_uri = f"azureml://{region}.api.azureml.ms/mlflow/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace}"

print(f"Update script with MLFlow Tracking URI: {mlflow_tracking_uri}")

## AML Submit Jobs
Submit a job to AML Compute and the custom environment.

In [ ]:
from azure.ai.ml import command, Input
import mltable

# Get the env definition
conda_env = ml_client.environments.get(name="workshop-conda-env", version="1")

# Get the mltable definition
titanic_version = 'xxxx'
data_asset = ml_client.data.get(name="titanic-cloud-example", version=titanic_version)

# define the command
command_job = command(
    code="./src",
    command="python main.py --input ${{inputs.mltable}} --mlflow_uri ${{inputs.mlflow_uri}}",
    environment=conda_env,
    inputs={
        "mltable": Input(type="mltable", path=data_asset.id),
        "mlflow_uri": mlflow_tracking_uri
    },

    ## cpu compute target created above
    compute=cpu_compute_target,
)

# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)

# get a URL for the status of the job
returned_job.studio_url

## AML Register Model
Once the model is trained, register the model for others to consume.


In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(returned_job.name),
    name="workshop-titanic-regression",
    description="Model created for workshop",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)